# Clasificación de tokens con HuggingFace

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../datos/procesamiento')
from corpus import Corpus
from etiquetado_entidades import codigo_hfl, split_rule, process_column

Cargando datos

In [2]:
main_db = pd.read_csv('../datos/DATA_HLF_MDS_2.csv',sep=',')

code_db = pd.read_excel('../datos/PRINCIPIOS_ACTIVOS_MDS.xlsx')
HLF = code_db.loc[:,['PRINCIPIO_ACTIVO','CODIGO_HLF']]
HLF_df = codigo_hfl(HLF)

df = main_db.join(HLF_df.set_index('CODIGO_MEDICAMENTO'), on='CODIGO_MEDICAMENTO')

Cargar principios activos, forma farma y juntar columnas

In [3]:
#PARA DETECTAR LA POSICION DE ENTIDADES TIPO PRINCIPIO ACTIVO, FORMA FARMACO SE COMPARARAN LOS ELEMENTOS DEL CORPUS CON LAS LISTAS CORRESPONDIENTES.
PA = np.unique(split_rule(df['PRINCIPIO_ACTIVO'].dropna().unique()))
FF = np.unique(split_rule(df['FORMA_FARMA'].dropna().unique()))

rows = (df['PRES_DENOMINACION'] + ' ' + df['RESUMEN']).dropna().unique()

Ejecutar procesamiento de datos de Martin

In [4]:
corpus = Corpus()

for i in range(len(rows)):
    row = rows[i].split()
    tagged_seq = process_column(row,PA,FF)

    corpus.append(tagged_seq)

    if i > 100:
        # solo testeando :)
        break

---

## Cargando el corpus como dataset de HuggingFace

In [5]:
ner_dict = {'O': 0,
            'B-ACTVPRNCP': 1,
            'I-ACTVPRNCP': 2,
            'B-ADMIN': 3,
            'I-ADMIN': 4,
            'B-PERIODICITY': 5,
            'I-PERIODICITY': 6,
            'B-DURATION': 7,
            'I-DURATION': 8
            }

corpus.entidades = ner_dict

In [6]:
HF_dataset = corpus.to_HF_dataset()

In [7]:
HF_dataset.data

InMemoryTable
id: int64
tokens: list<item: string>
  child 0, item: string
ner_tags: list<item: int64>
  child 0, item: int64
----
id: [[0,1,2,3,4,...,97,98,99,100,101]]
tokens: [[["PARACETAMOL","500","MG","COMPRIMIDO","1",...,"6","horas","durante","3","dias"],["KETOROLACO","10","MG","COMPRIMIDO","1",...,"8","horas","durante","3","dias"],...,["KETOPROFENO","50","MG","CÁPSULA","1",...,"ORAL","tres","veces","al","día"],["DICLOFENACO","75","MG/3","ML","SOLUCIÓN",...,"1","UNIDAD","INTRAMUSCULAR","dosis","simple"]]]
ner_tags: [[[1,0,0,3,0,...,6,6,7,8,8],[1,0,0,3,0,...,6,6,7,8,8],...,[1,0,0,3,0,...,3,0,0,0,0],[1,0,0,0,1,...,0,0,0,0,0]]]

In [8]:
HF_dataset.features[f"ner_tags"].feature.names = [key for key in ner_dict.keys()]
HF_dataset.features[f"ner_tags"].feature.names

['O',
 'B-ACTVPRNCP',
 'I-ACTVPRNCP',
 'B-ADMIN',
 'I-ADMIN',
 'B-PERIODICITY',
 'I-PERIODICITY',
 'B-DURATION',
 'I-DURATION']

In [9]:
HF_dataset = HF_dataset.train_test_split(test_size=0.2,seed=0)
HF_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 81
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 21
    })
})

## Cargamos un modelo basado en Transformers de HuggingFace

Nuestro modelo será: plncmm/bert-clinical-scratch-wl-es. Ha sido fine-tuneado con texto médico (aunque probablemente no con prescripciones)

Fuente de estasección ...

In [10]:
MODEL = "plncmm/bert-clinical-scratch-wl-es"

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)

Usaremos el tokenizador para codificar nuestro input.

In [12]:
example = HF_dataset["train"][0]

tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'meta',
 '##mi',
 '##zo',
 '##l',
 'só',
 '##dico',
 '1',
 'g',
 '/',
 '2',
 'ml',
 'solución',
 'inyecta',
 '##ble',
 'amp',
 '##olla',
 '2',
 'ml',
 '4',
 'unidad',
 'paren',
 '##tera',
 '##l',
 'diaria',
 '[SEP]']

Debemos corregir el _mismatch_ entre input tokenizado y la lista de tokens